# FungiMap: Exploratory Data Analysis

This notebook implements the exploratory data analysis phase of the FungiMap pipeline. The analysis includes the following steps:

1. **Sample Collection and Validation**: Quality control and metadata completeness assessment
2. **Read Quality Assessment**: FastQC analysis of raw sequencing data  
3. **Taxonomic Profiling**: Kraken2-based fungal content estimation
4. **Data Quality Filtering**: Sample selection based on fungal signal strength
5. **Resource Planning**: Computational resource estimation for downstream analysis
6. **Metadata Integration**: Sample annotation following community standards

The analysis prioritizes samples with:
- High metadata completeness (>70% of required fields)
- Sufficient fungal genomic content (>10% classified reads)
- Adequate sequencing depth (>1M read pairs)
- Low host contamination (<50% host-derived reads)

This exploratory phase generates validated sample manifests and resource allocation plans for the full FungiMap analysis pipeline.

In [ ]:
import os
import json
import hashlib
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from pysradb import SRAweb
import requests
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.stats as stats

# Set up plotting style
plt.style.use('seaborn')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

# Enable interactive plots in notebook
try:
    import IPython
    IPython.get_ipython().run_line_magic('matplotlib', 'inline')
except:
    pass

# Set up project paths
PROJECT_ROOT = Path('/Users/rohannorden/My Code/mycology-project')
DATA_DIR = PROJECT_ROOT / 'data'
RESULTS_DIR = PROJECT_ROOT / 'results'
CONFIG_DIR = PROJECT_ROOT / 'config'

# Create directories if they don't exist
for dir_path in [DATA_DIR, RESULTS_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

# Load configuration
with open(CONFIG_DIR / 'eda_config.json', 'r') as f:
    config = json.load(f)

# Set up custom visualization functions
def create_quality_heatmap(data, title):
    """Create an interactive quality heatmap."""
    fig = go.Figure(data=go.Heatmap(z=data))
    fig.update_layout(
        title=title,
        xaxis_title="Position in Read",
        yaxis_title="Quality Score",
        template="plotly_white"
    )
    return fig

def plot_read_quality_distribution(data, sample_name):
    """Plot read quality distribution with confidence intervals."""
    fig, ax = plt.subplots()
    sns.boxplot(data=data, ax=ax)
    sns.swarmplot(data=data, color=".25", size=2, ax=ax)
    ax.set_title(f"Read Quality Distribution - {sample_name}")
    ax.set_xlabel("Position in Read")
    ax.set_ylabel("Quality Score")
    return fig

def create_taxonomy_sunburst(taxa_data):
    """Create interactive sunburst plot for taxonomic data."""
    fig = px.sunburst(taxa_data, 
                      path=['kingdom', 'phylum', 'class', 'order'],
                      values='abundance',
                      title="Taxonomic Distribution")
    return fig

print("Environment setup complete. Configuration loaded.")
print("Advanced visualization functions initialized.")

## Data Harvesting Functions

The following cells implement functions to:
1. Query SRA/ENA/MGnify databases
2. Process and validate metadata
3. Check MIxS/ENVO compliance
4. Calculate data completeness scores

In [ ]:
class DataHarvester:
    def __init__(self, config):
        """Initialize data harvester with configuration."""
        self.config = config
        self.db = SRAweb()
        self.mgnify_api = "https://www.ebi.ac.uk/metagenomics/api/v1"
        self.manifest = []
        self.candidates = []
        
    def query_sra(self):
        """Query SRA database for metagenome and metatranscriptome datasets."""
        query_terms = self.config['databases']['sra']['query_terms']
        query = " OR ".join([f'"{term}"[All Fields]' for term in query_terms])
        query += ' AND ("metagenome"[Source] OR "metatranscriptome"[Source])'
        
        print(f"Querying SRA with: {query}")
        results = self.db.search(query)
        
        # Log the query in manifest
        self.manifest.append({
            'timestamp': datetime.now().isoformat(),
            'database': 'SRA',
            'query': query,
            'results_count': len(results)
        })
        
        return results
    
    def query_mgnify(self):
        """Query MGnify for fungal metagenomes."""
        filters = self.config['databases']['mgnify']['filters']
        params = {
            'experiment_type': filters['experiment_type'],
            'biome': ','.join(filters['biome'])
        }
        
        response = requests.get(f"{self.mgnify_api}/studies", params=params)
        data = response.json()
        
        # Log the query
        self.manifest.append({
            'timestamp': datetime.now().isoformat(),
            'database': 'MGnify',
            'query_params': params,
            'results_count': len(data.get('data', []))
        })
        
        return data
    
    def check_mixs_compliance(self, metadata):
        """Check MIxS/ENVO metadata completeness."""
        required_fields = self.config['output_formats']['metadata']
        present_fields = sum(1 for field in required_fields 
                           if field in metadata and metadata[field])
        return (present_fields / len(required_fields)) * 100
    
    def process_candidate(self, record):
        """Process a single candidate dataset."""
        metadata = record._asdict()
        
        # Calculate metadata completeness
        completeness = self.check_mixs_compliance(metadata)
        
        # Basic validation against thresholds
        read_pairs = int(metadata.get('spots', 0))
        meets_criteria = (
            read_pairs >= self.config['data_selection']['min_raw_read_pairs'],
            completeness >= self.config['data_selection']['min_metadata_completeness']
        )
        
        return {
            'accession': metadata.get('run_accession'),
            'habitat': metadata.get('sample_attribute', '').split(';')[0],
            'raw_read_pairs': read_pairs,
            'estimated_size_gb': float(metadata.get('size_MB', 0)) / 1024,
            'metadata_completeness': completeness,
            'metadata_url': f"https://www.ncbi.nlm.nih.gov/sra/{metadata.get('run_accession')}",
            'passes_criteria': all(meets_criteria)
        }
    
    def harvest_candidates(self):
        """Main method to harvest and process candidate datasets."""
        # Query databases
        sra_results = self.query_sra()
        mgnify_results = self.query_mgnify()
        
        # Process candidates
        print("Processing candidates...")
        for record in tqdm(sra_results.itertuples(), total=len(sra_results)):
            candidate = self.process_candidate(record)
            self.candidates.append(candidate)
        
        # Convert to DataFrame
        df = pd.DataFrame(self.candidates)
        
        # Save manifest
        manifest_df = pd.DataFrame(self.manifest)
        manifest_df.to_csv(RESULTS_DIR / 'manifest_draft.csv', index=False)
        
        return df

harvester = DataHarvester(config)
print("Data harvester initialized and ready.")

## Fetch and Process Candidate Accessions

Now we'll execute the data harvesting process and analyze the results. This will:
1. Query both SRA and MGnify databases
2. Process and validate the metadata
3. Generate initial candidate list
4. Filter based on our quality criteria

In [ ]:
# Create interactive scatter plot
fig = px.scatter(
    df_viz, 
    x='total_reads', 
    y='avg_length',
    color='env_broad_scale',
    size='metadata_completeness',
    hover_data=['sample_id', 'geo_loc_name', 'host'],
    title="FungiMap Dataset Overview",
    labels={
        'total_reads': 'Total Reads',
        'avg_length': 'Average Read Length (bp)',
        'env_broad_scale': 'Environment Type'
    },
    width=900,
    height=600
)

# Customize layout
fig.update_layout(
    title_x=0.5,
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.01
    ),
    margin=dict(r=150)
)

# Add sample size reference
fig.add_annotation(
    text="Bubble size = Metadata Completeness (%)",
    xref="paper", yref="paper",
    x=0.02, y=0.98,
    showarrow=False,
    font=dict(size=10, color="gray")
)

fig.show()

## Sample Processing Pipeline

Next, we'll implement the processing pipeline for our subsamples. This includes:
1. Downloading small subsamples (100k-500k reads)
2. Running QC checks
3. Performing taxonomic profiling
4. Calculating key statistics

In [ ]:
class SampleProcessor:
    def __init__(self, config):
        """Initialize sample processor with configuration."""
        self.config = config
        self.subsample_dir = DATA_DIR / 'subsamples'
        self.subsample_dir.mkdir(exist_ok=True)
        self.results = []
        self.qc_metrics = {}
        
    def fetch_subsample(self, accession):
        """Fetch a subsample of reads from SRA."""
        subsample_size = self.config['data_selection']['subsample_size']
        output_file = self.subsample_dir / f"{accession}_subsample.fastq"
        
        # Use fastq-dump to get subsample
        cmd = f"fastq-dump --split-spot --maxspot {subsample_size} "
        cmd += f"--outdir {self.subsample_dir} {accession}"
        
        # Execute command and log
        os.system(cmd)
        return output_file
    
    def parse_fastqc_data(self, fastqc_data_file):
        """Parse FastQC data file and return structured metrics."""
        metrics = {}
        current_section = None
        data = []
        
        with open(fastqc_data_file, 'r') as f:
            for line in f:
                line = line.strip()
                if line.startswith('>>'):
                    if current_section and data:
                        metrics[current_section] = pd.DataFrame(data)
                        data = []
                    current_section = line.split('\t')[0][2:]
                elif line.startswith('#'):
                    continue
                else:
                    data.append(line.split('\t'))
                    
        if current_section and data:
            metrics[current_section] = pd.DataFrame(data)
            
        return metrics
    
    def run_qc(self, fastq_file):
        """Run FastQC on subsample with enhanced metrics."""
        output_dir = self.subsample_dir / 'fastqc'
        output_dir.mkdir(exist_ok=True)
        
        cmd = f"fastqc -o {output_dir} {fastq_file}"
        os.system(cmd)
        
        # Parse FastQC results
        fastqc_data = self.parse_fastqc_data(
            output_dir / f"{fastq_file.stem}_fastqc/fastqc_data.txt"
        )
        
        # Calculate advanced QC metrics
        qc_metrics = {
            'basic_statistics': self._calculate_basic_stats(fastqc_data),
            'sequence_quality': self._analyze_sequence_quality(fastqc_data),
            'gc_content': self._analyze_gc_content(fastqc_data),
            'sequence_length': self._analyze_sequence_length(fastqc_data),
            'overrepresented': self._analyze_overrepresented(fastqc_data)
        }
        
        # Store QC metrics for visualization
        self.qc_metrics[fastq_file.stem] = qc_metrics
        
        return qc_metrics
    
    def _calculate_basic_stats(self, fastqc_data):
        """Calculate basic statistics from FastQC data."""
        if 'Basic Statistics' in fastqc_data:
            stats_df = fastqc_data['Basic Statistics']
            return dict(zip(stats_df[0], stats_df[1]))
        return {}
    
    def _analyze_sequence_quality(self, fastqc_data):
        """Analyze per base sequence quality."""
        if 'Per base sequence quality' in fastqc_data:
            qual_df = fastqc_data['Per base sequence quality']
            return {
                'mean_quality': float(qual_df[1].astype(float).mean()),
                'min_quality': float(qual_df[1].astype(float).min()),
                'quality_trend': qual_df[1].astype(float).tolist()
            }
        return {}
    
    def _analyze_gc_content(self, fastqc_data):
        """Analyze GC content distribution."""
        if 'Per sequence GC content' in fastqc_data:
            gc_df = fastqc_data['Per sequence GC content']
            gc_values = gc_df[0].astype(float)
            gc_counts = gc_df[1].astype(float)
            return {
                'mean_gc': float(np.average(gc_values, weights=gc_counts)),
                'gc_distribution': dict(zip(gc_values.tolist(), gc_counts.tolist()))
            }
        return {}
    
    def _analyze_sequence_length(self, fastqc_data):
        """Analyze sequence length distribution."""
        if 'Sequence Length Distribution' in fastqc_data:
            len_df = fastqc_data['Sequence Length Distribution']
            return {
                'length_range': f"{len_df[0].iloc[0]}-{len_df[0].iloc[-1]}",
                'distribution': dict(zip(len_df[0].tolist(), len_df[1].astype(float).tolist()))
            }
        return {}
    
    def _analyze_overrepresented(self, fastqc_data):
        """Analyze overrepresented sequences."""
        if 'Overrepresented sequences' in fastqc_data:
            over_df = fastqc_data['Overrepresented sequences']
            return {
                'num_overrepresented': len(over_df),
                'top_sequences': over_df.head(5).to_dict('records')
            }
        return {}
    
    def parse_kraken_report(self, report_file):
        """Parse Kraken2 report with enhanced taxonomic analysis."""
        taxa_data = []
        
        with open(report_file, 'r') as f:
            for line in f:
                parts = line.strip().split('\t')
                if len(parts) == 6:
                    percentage = float(parts[0])
                    reads = int(parts[1])
                    rank = parts[3]
                    taxid = parts[4]
                    name = parts[5].strip()
                    
                    taxa_data.append({
                        'percentage': percentage,
                        'reads': reads,
                        'rank': rank,
                        'taxid': taxid,
                        'name': name
                    })
        
        return pd.DataFrame(taxa_data)
    
    def run_kraken(self, fastq_file):
        """Run Kraken2 for taxonomic profiling with enhanced analysis."""
        output_prefix = self.subsample_dir / f"{fastq_file.stem}_kraken"
        output_file = f"{output_prefix}.txt"
        report_file = f"{output_prefix}_report.txt"
        
        cmd = f"kraken2 --db {self.config['qc_parameters']['kraken2']['db']} "
        cmd += f"--confidence {self.config['qc_parameters']['kraken2']['confidence']} "
        cmd += f"--output {output_file} --report {report_file} {fastq_file}"
        
        os.system(cmd)
        
        # Parse and analyze Kraken results
        taxa_df = self.parse_kraken_report(report_file)
        
        # Calculate advanced taxonomic metrics
        tax_metrics = {
            'classified_reads_percent': float(taxa_df['percentage'].sum()),
            'unique_taxa': len(taxa_df),
            'diversity_index': self._calculate_diversity_index(taxa_df),
            'top_taxa': taxa_df.nlargest(10, 'percentage')[['name', 'percentage', 'reads']].to_dict('records'),
            'rank_distribution': taxa_df.groupby('rank')['percentage'].sum().to_dict()
        }
        
        return tax_metrics
    
    def _calculate_diversity_index(self, taxa_df):
        """Calculate Shannon diversity index from taxonomic data."""
        proportions = taxa_df['percentage'] / 100
        proportions = proportions[proportions > 0]  # Remove zeros
        return float(-np.sum(proportions * np.log(proportions)))
    
    def process_sample(self, accession):
        """Process a single sample through the pipeline with enhanced analysis."""
        try:
            # Fetch subsample
            fastq_file = self.fetch_subsample(accession)
            
            # Run QC with enhanced metrics
            qc_results = self.run_qc(fastq_file)
            
            # Run Kraken with enhanced taxonomic analysis
            tax_results = self.run_kraken(fastq_file)
            
            # Compile comprehensive results
            results = {
                'accession': accession,
                'qc_metrics': qc_results,
                'taxonomic_profile': tax_results,
                'quality_score': self._calculate_quality_score(qc_results, tax_results),
                'processing_completed': True
            }
            
            self.results.append(results)
            return results
            
        except Exception as e:
            print(f"Error processing {accession}: {str(e)}")
            return None
    
    def _calculate_quality_score(self, qc_results, tax_results):
        """Calculate overall quality score based on multiple metrics."""
        scores = []
        
        # Sequence quality score (0-100)
        if 'sequence_quality' in qc_results:
            mean_qual = qc_results['sequence_quality']['mean_quality']
            scores.append(min(100, mean_qual * 2.5))  # Scale quality to 0-100
            
        # GC content score (0-100)
        if 'gc_content' in qc_results:
            gc_mean = qc_results['gc_content']['mean_gc']
            gc_score = 100 - abs(gc_mean - 50) * 2  # Penalize deviation from 50%
            scores.append(max(0, gc_score))
            
        # Taxonomic classification score (0-100)
        classified_pct = tax_results['classified_reads_percent']
        scores.append(min(100, classified_pct))
        
        # Diversity score (0-100)
        diversity = tax_results['diversity_index']
        scores.append(min(100, diversity * 20))  # Scale diversity to 0-100
        
        return float(np.mean(scores))

# Initialize processor with enhanced capabilities
processor = SampleProcessor(config)
print("Enhanced sample processor initialized with advanced metrics and visualization support.")

## Process Initial Subset of Samples

We'll now process a small subset of our candidates to:
1. Validate our pipeline
2. Generate initial QC metrics
3. Estimate full processing requirements

In [ ]:
# Select initial subset for processing with enhanced visualization
pilot_size = 5  # Start with 5 samples for initial testing
pilot_samples = candidates_df[candidates_df['passes_criteria']].head(pilot_size)

# Process pilot samples
print(f"Processing {pilot_size} pilot samples...")
pilot_results = []
for _, sample in tqdm(pilot_samples.iterrows(), total=pilot_size):
    result = processor.process_sample(sample['accession'])
    if result:
        pilot_results.append(result)

# Convert results to DataFrame
pilot_df = pd.DataFrame(pilot_results)

# Create comprehensive visualization dashboard
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=(
        'Quality Score Distribution',
        'Taxonomic Classification',
        'GC Content Distribution',
        'Read Length Distribution',
        'Per-Base Quality Heatmap',
        'Diversity Index by Sample'
    ),
    vertical_spacing=0.12
)

# 1. Quality Score Distribution
quality_scores = [r['quality_score'] for r in pilot_results]
fig.add_trace(
    go.Box(y=quality_scores, name='Quality Scores',
           boxpoints='all', jitter=0.3, pointpos=-1.8),
    row=1, col=1
)

# 2. Taxonomic Classification
for i, result in enumerate(pilot_results):
    top_taxa = pd.DataFrame(result['taxonomic_profile']['top_taxa'])
    fig.add_trace(
        go.Bar(x=top_taxa['name'], y=top_taxa['percentage'],
               name=result['accession']),
        row=1, col=2
    )

# 3. GC Content Distribution
for i, result in enumerate(pilot_results):
    gc_data = result['qc_metrics']['gc_content']['gc_distribution']
    fig.add_trace(
        go.Scatter(x=list(gc_data.keys()), y=list(gc_data.values()),
                  name=result['accession'], mode='lines'),
        row=2, col=1
    )

# 4. Read Length Distribution
for i, result in enumerate(pilot_results):
    length_data = result['qc_metrics']['sequence_length']['distribution']
    fig.add_trace(
        go.Scatter(x=list(length_data.keys()), y=list(length_data.values()),
                  name=result['accession'], mode='lines'),
        row=2, col=2
    )

# 5. Per-Base Quality Heatmap
quality_trends = np.array([r['qc_metrics']['sequence_quality']['quality_trend'] 
                          for r in pilot_results])
fig.add_trace(
    go.Heatmap(z=quality_trends, 
               colorscale='Viridis',
               showscale=True),
    row=3, col=1
)

# 6. Diversity Index
diversity_scores = [r['taxonomic_profile']['diversity_index'] for r in pilot_results]
accessions = [r['accession'] for r in pilot_results]
fig.add_trace(
    go.Bar(x=accessions, y=diversity_scores,
           name='Shannon Diversity Index'),
    row=3, col=2
)

# Update layout
fig.update_layout(
    height=1200,
    showlegend=True,
    title_text="Enhanced Sample Quality Analysis Dashboard"
)

# Show the plot
fig.show()

# Generate comprehensive statistical summary
print("\nPilot Processing Results Summary:")
print("================================")
print(f"Successfully processed: {len(pilot_df)}/{pilot_size}")

print("\nQuality Metrics Summary:")
print("----------------------")
quality_summary = pd.DataFrame([{
    'Metric': 'Overall Quality Score',
    'Mean': np.mean(quality_scores),
    'Std': np.std(quality_scores),
    'Min': np.min(quality_scores),
    'Max': np.max(quality_scores)
}])
print(quality_summary.to_string(index=False))

print("\nTaxonomic Classification Summary:")
print("-------------------------------")
for result in pilot_results:
    print(f"\nSample: {result['accession']}")
    print(f"Classified Reads: {result['taxonomic_profile']['classified_reads_percent']:.2f}%")
    print(f"Unique Taxa: {result['taxonomic_profile']['unique_taxa']}")
    print(f"Diversity Index: {result['taxonomic_profile']['diversity_index']:.2f}")
    print("\nTop 5 Taxa:")
    for taxon in result['taxonomic_profile']['top_taxa'][:5]:
        print(f"  {taxon['name']:50}: {taxon['percentage']:6.2f}%")

# Save enhanced pilot results
output_file = RESULTS_DIR / 'enhanced_pilot_analysis.json'
with open(output_file, 'w') as f:
    # Convert numpy types to native Python types for JSON serialization
    serializable_results = []
    for result in pilot_results:
        serializable = {
            'accession': result['accession'],
            'quality_score': float(result['quality_score']),
            'taxonomic_profile': {
                k: (float(v) if isinstance(v, np.float32) else v)
                for k, v in result['taxonomic_profile'].items()
            },
            'qc_metrics': result['qc_metrics']
        }
        serializable_results.append(serializable)
    json.dump(serializable_results, f, indent=2)

print(f"\nEnhanced analysis results saved to {output_file}")
print("\nInteractive visualization dashboard generated with comprehensive metrics.")

## Resource Estimation

Based on our pilot processing, we'll now:
1. Calculate storage requirements
2. Estimate compute needs
3. Project total processing time
4. Generate cost estimates

In [ ]:
# Calculate resource requirements
def estimate_resources(pilot_df, full_dataset_size):
    """Estimate resource requirements based on pilot results."""
    # Storage estimates
    avg_sample_size = candidates_df['estimated_size_gb'].mean()
    total_storage_gb = avg_sample_size * full_dataset_size
    
    # Compute time estimates (based on pilot processing times)
    avg_processing_time = 0.5  # hours per sample (placeholder - implement actual timing)
    total_processing_time = avg_processing_time * full_dataset_size
    
    # Cost estimates (placeholder values - adjust based on your cloud provider)
    storage_cost_per_gb = 0.02  # $/GB/month
    compute_cost_per_hour = 1.0  # $/hour
    
    estimates = {
        'storage': {
            'total_gb': total_storage_gb,
            'cost_per_month': total_storage_gb * storage_cost_per_gb
        },
        'compute': {
            'total_hours': total_processing_time,
            'cost': total_processing_time * compute_cost_per_hour
        },
        'total_cost_estimate': (total_storage_gb * storage_cost_per_gb + 
                              total_processing_time * compute_cost_per_hour)
    }
    
    return estimates

# Calculate estimates for full PoC (30 samples) and expanded pilot (15 samples)
poc_estimates = estimate_resources(pilot_df, 30)
pilot_estimates = estimate_resources(pilot_df, 15)

# Save estimates
estimates = {
    'full_poc': poc_estimates,
    'expanded_pilot': pilot_estimates,
    'calculation_timestamp': datetime.now().isoformat()
}

with open(RESULTS_DIR / 'estimated_costs.json', 'w') as f:
    json.dump(estimates, f, indent=2)

# Print summary
print("Resource Estimates:")
print("\nExpanded Pilot (15 samples):")
print(f"Storage: {pilot_estimates['storage']['total_gb']:.1f} GB")
print(f"Processing Time: {pilot_estimates['compute']['total_hours']:.1f} hours")
print(f"Estimated Cost: ${pilot_estimates['total_cost_estimate']:.2f}")

print("\nFull PoC (30 samples):")
print(f"Storage: {poc_estimates['storage']['total_gb']:.1f} GB")
print(f"Processing Time: {poc_estimates['compute']['total_hours']:.1f} hours")
print(f"Estimated Cost: ${poc_estimates['total_cost_estimate']:.2f}")

## Generate Final Reports and Recommendations

Now we'll:
1. Generate the QC report
2. Create the recommended expanded pilot list
3. Prepare final deliverables

In [ ]:
# Select samples for expanded pilot
def select_expanded_pilot(candidates_df, pilot_df, n_samples=12, n_backups=6):
    """Select samples for expanded pilot based on our criteria."""
    # Combine metrics from candidates and pilot results
    merged_df = candidates_df.merge(pilot_df, on='accession', how='left')
    
    # Score each sample
    merged_df['total_score'] = (
        merged_df['metadata_completeness'] * 0.4 +
        merged_df['fungal_percent'].fillna(0) * 0.4 +
        (merged_df['passes_criteria'].astype(int) * 100) * 0.2
    )
    
    # Sort by score and select top samples
    recommended = merged_df.nlargest(n_samples, 'total_score')
    backups = merged_df[~merged_df['accession'].isin(recommended['accession'])]
    backups = backups.nlargest(n_backups, 'total_score')
    
    # Prepare recommendation DataFrame
    recommendations = pd.concat([
        recommended.assign(status='recommended'),
        backups.assign(status='backup')
    ])
    
    recommendations['rationale'] = recommendations.apply(
        lambda x: f"Metadata: {x['metadata_completeness']:.1f}%, "
                 f"Fungal: {x.get('fungal_percent', 'N/A')}%, "
                 f"Size: {x['estimated_size_gb']:.1f}GB",
        axis=1
    )
    
    return recommendations

# Generate recommendations
expanded_pilot = select_expanded_pilot(candidates_df, pilot_df)
expanded_pilot.to_csv(RESULTS_DIR / 'recommended_expanded_pilot.csv', index=False)

# Generate HTML report
def generate_qc_report():
    """Generate QC report in HTML format."""
    import plotly.express as px
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go
    
    # Create report
    with open(RESULTS_DIR / 'QC_report.html', 'w') as f:
        # Write header
        f.write("""
        <html>
        <head>
            <title>MycoGraph-XL EDA Results</title>
            <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css" rel="stylesheet">
        </head>
        <body>
        <div class="container mt-5">
        <h1>MycoGraph-XL: Exploratory Data Analysis Report</h1>
        """)
        
        # Add summary statistics
        f.write(f"""
        <h2>Summary Statistics</h2>
        <ul>
            <li>Total candidates: {len(candidates_df)}</li>
            <li>Passing all criteria: {candidates_df['passes_criteria'].sum()}</li>
            <li>Average metadata completeness: {candidates_df['metadata_completeness'].mean():.1f}%</li>
        </ul>
        """)
        
        # Add visualizations
        # (In practice, add plotly figures here)
        
        f.write("</div></body></html>")

# Generate reports
generate_qc_report()

print("\nFinal Deliverables Generated:")
print("1. recommended_expanded_pilot.csv")
print("2. QC_report.html")
print("3. estimated_costs.json")
print("\nRecommended next steps:")
print("1. Review expanded pilot recommendations")
print("2. Validate resource estimates")
print("3. Proceed with expanded pilot upon approval")